<a href="https://colab.research.google.com/github/rfcx/rfcx-guardian-cli/blob/master/notebooks/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tensorflow==2.0.0b1

     |████████████████████████████████| 87.9MB 37kB/s 
     |████████████████████████████████| 3.1MB 42.3MB/s 
     |████████████████████████████████| 501kB 54.2MB/s 
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
import re

print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

2.0.0-beta1


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [5]:
from google.colab import drive
drive_mountpoint = '/content/data/'
drive.mount(drive_mountpoint)
%cd $drive_mountpoint/My\ Drive/Colab\ Notebooks
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/data/
/content/data/My Drive/Colab Notebooks
 1-DatasetPrep.ipynb                          hitachi_nuttov3.tflite
'2-Train backup.ipynb'                        hitachi_test_set/
'2-Train model.ipynb'                         hitachi_train_set/
'2-Train model (tflite)'                      kaggle2018_meow_bark_laugh/
 34fb7822-0ca6-4002-a233-9072c8c42607.wav     kaggle2018_meow_bark_laugh.tflite
 3a46e1f5-f1d0-41f1-b771-64154be74f96.0.wav   kaggle2018_train/
'3-Test m

In [6]:
#@title Insert train dataset path and test data set path

train_dataset_directory = "gunshot_dataset/train_set/" #@param {type:"string"}
test_dataset_directory = "gunshot_dataset/test_set/" #@param {type:"string"}
train_classes = os.listdir(train_dataset_directory)
test_classes = os.listdir(test_dataset_directory)
print('Found classes in train: ' + ', '.join(train_classes))
print('Found classes in test: ' + ', '.join(test_classes))

train_dataset = list()
test_dataset = list()

for c in train_classes:
  train_dataset.extend([[c+'/'+x, c] for x in os.listdir(train_dataset_directory + c) if re.search("\.[0-9]+\.wav$", x)])

for c in test_classes:
  test_dataset.extend([[c+'/'+x, c] for x in os.listdir(test_dataset_directory + c) if re.search("\.[0-9]+\.wav$", x)])

print('Examples train dataset:', train_dataset[1600:0:-100])
print('Examples test dataset:', test_dataset[10:1000:100])
print('Size of train dataset', len(train_dataset))
print('Size of test dataset', len(test_dataset))
print('Total size of dataset', len(train_dataset) + len(test_dataset))

Found classes in train: gunshot, environment
Found classes in test: gunshot, environment
Examples train dataset: [['environment/Jk2B7Mvh6cE.109.wav', 'environment'], ['gunshot/dqJWTzC3s8I.106.wav', 'gunshot'], ['gunshot/cKK0oZrwed8.33.wav', 'gunshot'], ['gunshot/bciLCctZfjE.101.wav', 'gunshot'], ['gunshot/7RlyBtFDd98.78.wav', 'gunshot'], ['gunshot/3PfWjclBA5I.16.wav', 'gunshot'], ['gunshot/--aaILOrkII.103.wav', 'gunshot'], ['gunshot/YjNQ5TGRChs.30.wav', 'gunshot'], ['gunshot/wdVYP2ijnew.101.wav', 'gunshot'], ['gunshot/u_Nxu8TvonA.101.wav', 'gunshot'], ['gunshot/SLAmK__qyXM.15.wav', 'gunshot'], ['gunshot/QH3RIg1ypDg.29.wav', 'gunshot'], ['gunshot/nWlDkrIzc0g.107.wav', 'gunshot'], ['gunshot/l6HsxUNdkd4.17.wav', 'gunshot'], ['gunshot/JfQZeAIHg3s.70.wav', 'gunshot'], ['gunshot/hmD1w6133Og.102.wav', 'gunshot']]
Examples test dataset: [['gunshot/2ffgd7k0vI8.45.wav', 'gunshot'], ['gunshot/Dj9gyAoqmQ0.35.wav', 'gunshot'], ['gunshot/M7rGKqfZh6k.31.wav', 'gunshot'], ['gunshot/UN84r_4GTdQ.27.wav'

In [7]:
# read_wav function for reading raw audio data
from importlib.machinery import SourceFileLoader
read_wav = SourceFileLoader('ReadWav' , 'services/AudioIference/AudioInference.py').load_module().ReadWav

# test the read_wav function
waveform, source_sample_rate = read_wav(train_dataset_directory + train_dataset[0][0])
print(source_sample_rate)
print(waveform.size)

12000
24000


In [0]:
def make_x_sample(record, data_dir):
  path = data_dir + record[0]
  waveform, _ = read_wav(path)
  return waveform[:, np.newaxis]

def make_y_sample(record):
  return 0 if record[1] == 'environment' else 1

train_x = np.array([make_x_sample(x, train_dataset_directory) for x in train_dataset], dtype=np.float32)
train_y = np.array([make_y_sample(y) for y in train_dataset], dtype=np.int64)
test_x = np.array([make_x_sample(x, test_dataset_directory) for x in test_dataset], dtype=np.float32)
test_y = np.array([make_y_sample(y) for y in test_dataset], dtype=np.int64)

In [0]:
batch_size = 53
context_duration_seconds = 2.0
sample_rate = 12000 # relate to audio sample rate
number_of_classes = len(train_classes)

context_duration_samples = int(context_duration_seconds * sample_rate)
context_step_samples = context_duration_samples // 2
  
@tf.function
def front_end(waveform):

  # Remove channels
  #mono = tf.reduce_mean(waveform, axis=1)
  mono = tf.squeeze(waveform, 2)
  
  # Generate spectrograms
  full_spectrograms = tf.signal.stft(
      mono, frame_length=1024, frame_step=256, fft_length=1024)
  magnitude_spectrograms = tf.math.abs(full_spectrograms)
  log_spectrograms = tf.math.log(magnitude_spectrograms + 1e-6)
 
  # Put the channel back in
  result = tf.expand_dims(log_spectrograms, 3)

  return result

In [10]:
from tensorflow import keras

def res_net_block(input_data, filters, conv_size):
  x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(filters, conv_size, activation=None, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Add()([x, input_data])
  x = layers.Activation('relu')(x)
  return x

inputs = keras.Input(shape=(24000, 1))
spectrogram = layers.Lambda(front_end)(inputs)
x = layers.Conv2D(32, 3, activation='relu')(spectrogram)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)
num_res_net_blocks = 10
for i in range(num_res_net_blocks):
    x = res_net_block(x, 64, 3)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)
res_net_model = keras.Model(inputs, outputs)

res_net_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 24000, 1)]   0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 90, 513, 1)   0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 88, 511, 32)  320         lambda[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 86, 509, 64)  18496       conv2d[0][0]                     
______________________________________________________________________________________________

In [0]:
res_net_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
res_net_model.fit(train_x, train_y, validation_split=0.1, epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 3135 samples, validate on 349 samples
Epoch 1/10
1824/3135 [================>.............] - ETA: 20:59 - loss: 0.9037 - accuracy: 0.5433

In [0]:
evaluate_value = model.evaluate(test_x, test_y, batch_size = batch_size, verbose = 1)

print('Evaluation value: {}'.format(evaluate_value))